In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import requests
import pymysql
from pymysql import IntegrityError, InterfaceError
from selenium.common.exceptions import NoSuchElementException

In [3]:
# 네이버 증권에 접속 -> 해당 기업의 산업 코드 얻기
def get_category(code):
    try:
        url = f"https://finance.naver.com/item/main.naver?code={code}"
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        category = soup.select_one('div.trade_compare>h4>em>a').text
        return category

    except:
        print("산업 분야를 찾지 못함")
        return

In [4]:
#pymysql 커서를 띄워 놓음 - 닫기는 db.close()

In [5]:

get_name = re.compile(".+?(?=\()") # .+만 쓰면 가장 큰 범위부터 탐색해서 매칭하면 끝남. ?을 붙여 reluctant를 이용한다.
get_code = re.compile("([0-9]{6})")
today = datetime.today().date()

db = pymysql.connect(host='localhost', user = 'root', passwd = '0000', db = 'project02', charset = 'utf8')

# 나중에 db에 데이터 저장할 때 밑에 db.commit(), db.close() 달아주자 / commit()없이는 저장안됨
with db.cursor() as curs:
    with webdriver.Chrome('./chromedriver.exe') as driver:
        driver.implicitly_wait(2)
        for i in range(1, 3):
            
            # 최초 실행 : 5월 1일부터 오늘 날짜까지 긁어옴
            driver.get(f'http://hkconsensus.hankyung.com/apps.analysis/analysis.list?sdate=2022-05-01&edate={today}&now_page={i}&search_value=&report_type=UP&pagenum=80&search_text=&business_code=')

            # 날마다 자동 실행 구현 필요 : 서버에 올리면 스크립트만 주면 될 것 같은데 로컬에 있으면 파일을 켜야 하지 않을까?
            # driver.get(f'http://hkconsensus.hankyung.com/apps.analysis/analysis.list?sdate={start_day}&edate={today}&now_page={i}&search_value=&report_type=UP&pagenum=80&search_text=&business_code=')
            

            contents = driver.find_elements(By.CSS_SELECTOR, 'table>tbody>tr')
            
            for j in contents:
                # 웹 스크래핑
                try:
                    
                    date = j.find_element(By.CSS_SELECTOR, 'td.txt_number').text
                    report = j.find_element(By.CSS_SELECTOR, 'td.text_l').text
                    name = get_name.findall(report)[0]
                    code = get_code.findall(report)[0]
                    category = get_category(code) 
                    made_by = j.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text


                    print(name, code, date, category, made_by) 
                    
                # pymysql에 저장
                    query1 = "INSERT INTO reports(company, date, written_by) VALUES(%s, %s, %s);"
                    curs.execute(query1, (name, date, made_by))

                    query2 = "INSERT INTO companies(company, code, category) VALUES(%s, %s, %s);"
                    curs.execute(query2, (name, code, category))
                    
                    
                except InterfaceError:
                    print("인터페이스 에러 발생")

                except IntegrityError as i:
                    pass
                
                except NoSuchElementException:
                    break # 이거 되는지 싫험좀 해봄 - 공란 만나면 다음 페이지로 넘어가게 - 되는 듯!

db.commit()
db.close()

C:\Users\dowra\AppData\Local\Temp\ipykernel_16524\3269302814.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome('./chromedriver.exe') as driver:


카카오 035720 2022-08-05 양방향미디어와서비스 이베스트증권
카카오게임즈 293490 2022-08-04 게임엔터테인먼트 이베스트증권
제이브이엠 054950 2022-08-03 건강관리장비와용품 한화투자증권
한국타이어앤테크놀로지 161390 2022-08-03 자동차부품 대신증권
한국타이어앤테크놀로지 161390 2022-08-03 자동차부품 메리츠증권
(1062, "Duplicate entry '한국타이어앤테크놀로지' for key 'companies.PRIMARY'")
F&F 383220 2022-08-02 섬유,의류,신발,호화품 대신증권
만도 204320 2022-08-02 자동차부품 메리츠증권
F&F 383220 2022-08-02 섬유,의류,신발,호화품 메리츠증권
(1062, "Duplicate entry 'F&F' for key 'companies.PRIMARY'")
한화에어로스페이스 012450 2022-08-01 우주항공과국방 유진투자증권
현대중공업 329180 2022-08-01 조선 메리츠증권
현대미포조선 010620 2022-08-01 조선 메리츠증권
만도 204320 2022-08-01 자동차부품 메리츠증권
(1062, "Duplicate entry '만도' for key 'companies.PRIMARY'")
대웅제약 069620 2022-07-29 제약 SK증권
한화솔루션 009830 2022-07-29 화학 SK증권
고려아연 010130 2022-07-29 비철금속 하이투자증권
제이브이엠 054950 2022-07-29 건강관리장비와용품 이베스트증권
(1062, "Duplicate entry '제이브이엠' for key 'companies.PRIMARY'")
대웅제약 069620 2022-07-29 제약 이베스트증권
(1062, "Duplicate entry '대웅제약' for key 'companies.PRIMARY'")
SK바이오사이언스 302440 2022-07-29 제약 이베스트증권
LG이노텍 011070 2022-

In [22]:

with webdriver.Chrome('./chromedriver.exe') as driver:
    for i in range(1, 3):
        driver.get(f'http://hkconsensus.hankyung.com/apps.analysis/analysis.list?sdate=2022-05-01&edate={today}&now_page={i}&search_value=&report_type=UP&pagenum=80&search_text=&business_code=')

        contents = driver.find_elements(By.CSS_SELECTOR, 'table>tbody>tr')
        
        for j in contents:
            print(j.find_element(By.CSS_SELECTOR, 'td:first-child').text)

C:\Users\dowra\AppData\Local\Temp\ipykernel_16936\373494234.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome('./chromedriver.exe') as driver:


2022-08-05
2022-08-04
2022-08-03
2022-08-03
2022-08-03
2022-08-02
2022-08-02
2022-08-02
2022-08-01
2022-08-01
2022-08-01
2022-08-01
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-29
2022-07-28
2022-07-28
2022-07-27
2022-07-27
2022-07-25
2022-07-25
2022-07-22
2022-07-22
2022-07-22
2022-07-19
2022-07-18
2022-07-14
2022-07-13
2022-07-12
2022-07-11
2022-07-11
2022-07-06
2022-07-05
2022-07-05
2022-07-05
2022-07-05
2022-07-05
2022-07-05
2022-06-29
2022-06-29
2022-06-27
2022-06-21
2022-06-21
2022-06-21
2022-06-17
2022-06-17
2022-06-15
2022-06-15
2022-06-14
2022-06-13
2022-06-10
2022-06-08
2022-06-08
2022-06-03
2022-06-02
2022-06-02
2022-06-02
2022-06-02
2022-06-02
2022-05-31
2022-05-31
2022-05-31
2022-05-31
2022-05-31
2022-05-31
2022-05-31
2022-05-31
2022-05-30
2022-05-30
2022-05-30
2022-05-30










2022-05-30
2022-05-27
2022-05-27
2022-05-26
2022-05-26
2022-05-26
2022-05-26
2022-05-25
2022-05-23
2022-05-20
